In [7]:
import MeCab
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pymysql
import sqlalchemy
import math
import re
import ast
from gensim.models import Word2Vec
import math

In [8]:
df = pd.read_csv('esg_report_2018_19_21_cleansing.csv')
df['noun']  = df['noun'].map(lambda x : ast.literal_eval(x))
df.head(5)

,filename,content,noun
0,report_2018,2018 SUSTINVEST,[SUSTINVEST]
1,report_2018,상장기업 ESG 분석 보고서,"[상장, 기업, ESG, 분석, 보고서]"
2,report_2018,1 글로벌 책임투자 현황,"[글로벌, 책임, 투자, 현황]"
3,report_2018,2 국내 책임투자 현황,"[국내, 책임, 투자, 현황]"
4,report_2018,1 평가대상기업 및 기본정보,"[평가, 대상, 기업, 기본, 정보]"


In [9]:
tokenized_data = []
for noun_list in df['noun']:
    tokenized_data.append(noun_list)
tokenized_data[:10]

[['SUSTINVEST'],
 ['상장', '기업', 'ESG', '분석', '보고서'],
 ['글로벌', '책임', '투자', '현황'],
 ['국내', '책임', '투자', '현황'],
 ['평가', '대상', '기업', '기본', '정보'],
 ['ESG', '성과', '우수', '기업'],
 ['V', 'ESG', 'MOMENTUM', '기업'],
 ['ESG', 'MOMENTUM', '기업'],
 ['VI', 'ESG', 'CONTROVERSY', '기업'],
 ['ESG', 'CONTROVERSY', '기업']]

In [10]:
count_table = pd.DataFrame(columns = ['noun','count','total','probability'])
pmi_table = pd.DataFrame(columns = ['noun1','noun2','co-occurence','pmi'])

In [11]:
corp = tokenized_data
listb = []

dic={}
tot=0

dic2d={}
for d in tqdm(corp):
    for w in d:
        tot+=1
        if w not in dic:
            dic[w]=1
        else:
            dic[w]+=1

    l=len(d)
    for i1 in range(0,l-1):
        for i2 in range(i1+1, l):
            w1=d[i1]
            w2=d[i2]
            if w1 not in dic2d:
                dic2d[w1]={}
            if w2 not in dic2d[w1]:
                dic2d[w1][w2]=1
            else:
                dic2d[w1][w2]+=1

            if w2 not in dic2d:
                dic2d[w2]={}
            if w1 not in dic2d[w2]:
                dic2d[w2][w1]=1
            else:
                dic2d[w2][w1]+=1

for key in tqdm(dic.keys()):
    pr = dic[key]/tot
    #print(key, dic[key], tot, pr)
    aa = {'noun1' : key, 
          'count' : dic[key],
          'total' : tot,
          'probability' : pr}
    #count_table.append(aa)
    #print('-'*50)


pmi={}
for key1 in tqdm(dic2d.keys()):
    dsub=dic2d[key1]
    pmi[key1]={}
    for key2 in dsub.keys():
        pmiValue=math.log((dic2d[key1][key2]/tot)/((dic[key1]/tot)*(dic[key2]/tot))) #pmi 수식(값이 높을수록 관련성이 높다.)
        #print(key1, key2, dic2d[key1][key2], pmiValue)
        bb = {'noun1' : key1, 
              'noun2' : key2,
              'co-occurence' : dic2d[key1][key2],
              'pmi' : pmiValue}
        listb.append(bb)
        
        pmi[key1][key2]=pmiValue

  0%|          | 0/2126 [00:00<?, ?it/s]

  0%|          | 0/5042 [00:00<?, ?it/s]

  0%|          | 0/5040 [00:00<?, ?it/s]

In [18]:
# 단순 카운트
dic

{'SUSTINVEST': 1,
 '상장': 73,
 '기업': 921,
 'ESG': 462,
 '분석': 52,
 '보고서': 116,
 '글로벌': 32,
 '책임': 168,
 '투자': 236,
 '현황': 59,
 '국내': 117,
 '평가': 710,
 '대상': 206,
 '기본': 34,
 '정보': 183,
 '성과': 293,
 '우수': 45,
 'V': 5,
 'MOMENTUM': 4,
 'VI': 3,
 'CONTROVERSY': 12,
 '주요': 85,
 '이슈': 221,
 '환경': 388,
 'E': 50,
 '탄소': 35,
 '배출권': 22,
 '거래제': 14,
 '통합': 45,
 '관리': 408,
 '제도': 110,
 '사회': 352,
 'S': 41,
 '문재': 3,
 '정부': 84,
 '계속': 20,
 '소득': 5,
 '주도': 14,
 '성장': 43,
 '지배': 200,
 '구조': 232,
 'G': 47,
 '감시': 19,
 '강화': 92,
 '공정': 153,
 '경제': 37,
 '실현': 15,
 'Responsible': 2,
 'Investment': 8,
 'RI': 2,
 '의사': 26,
 '결정': 74,
 '시': 65,
 '요소': 28,
 '고려': 63,
 '기법': 8,
 '재무': 46,
 '리스크': 161,
 '효과': 47,
 '지속': 228,
 '가능': 229,
 '창출': 13,
 '목적': 24,
 '세계': 26,
 '규모': 243,
 '증가': 106,
 '기간': 95,
 '동안': 47,
 '기준': 245,
 '한': 5,
 '운용': 85,
 '자산': 113,
 '대비': 106,
 '차지': 40,
 '스튜어드': 43,
 '도입': 89,
 '계기': 12,
 'GPIF': 1,
 '하': 3,
 '적극': 30,
 '전파': 1,
 '일본': 7,
 '성': 11,
 '장': 34,
 '호주': 3,
 '뉴질랜드': 2,
 '

In [13]:
report_esg_pmi_result = []
for i in range(len(listb)) :
    if listb[i]['noun1'] == 'ESG' :
        report_esg_pmi_result .append(listb[i])
print(report_esg_pmi_result )

[{'noun1': 'ESG', 'noun2': '상장', 'co-occurence': 29, 'pmi': 3.8763230042074537}, {'noun1': 'ESG', 'noun2': '기업', 'co-occurence': 702, 'pmi': 4.527959983139875}, {'noun1': 'ESG', 'noun2': '분석', 'co-occurence': 49, 'pmi': 4.74006319489857}, {'noun1': 'ESG', 'noun2': '보고서', 'co-occurence': 58, 'pmi': 4.106339434809425}, {'noun1': 'ESG', 'noun2': '성과', 'co-occurence': 287, 'pmi': 4.778796222111924}, {'noun1': 'ESG', 'noun2': '우수', 'co-occurence': 54, 'pmi': 4.981808172163325}, {'noun1': 'ESG', 'noun2': 'V', 'co-occurence': 5, 'pmi': 4.7994866153693705}, {'noun1': 'ESG', 'noun2': 'MOMENTUM', 'co-occurence': 4, 'pmi': 4.7994866153693705}, {'noun1': 'ESG', 'noun2': 'VI', 'co-occurence': 3, 'pmi': 4.7994866153693705}, {'noun1': 'ESG', 'noun2': 'CONTROVERSY', 'co-occurence': 12, 'pmi': 4.7994866153693705}, {'noun1': 'ESG', 'noun2': '주요', 'co-occurence': 30, 'pmi': 3.7580327405412097}, {'noun1': 'ESG', 'noun2': '이슈', 'co-occurence': 153, 'pmi': 4.431761835244053}, {'noun1': 'ESG', 'noun2': '평가',

In [14]:
report_esg_pmi_data = pd.DataFrame(report_esg_pmi_result)

In [15]:
report_esg_pmi_data

,noun1,noun2,co-occurence,pmi
0,ESG,상장,29,3.876323
1,ESG,기업,702,4.527960
2,ESG,분석,49,4.740063
3,ESG,보고서,58,4.106339
4,ESG,성과,287,4.778796
...,...,...,...,...
1333,ESG,실효,1,1.966273
1334,ESG,구성원,1,2.720045
1335,ESG,내재,1,3.413192
1336,ESG,미해결,1,4.799487


In [17]:
report_esg_pmi_data_sort = report_esg_pmi_data.sort_values(by=['pmi'], ascending=[False])
report_esg_pmi_data_sort = report_esg_pmi_data_sort.reset_index()

report_esg_pmi_data_sort.pop("index")
report_esg_pmi_data_sort

,noun1,noun2,co-occurence,pmi
0,ESG,향조,9,6.996711
1,ESG,출하,8,6.878928
2,ESG,정한,8,6.878928
3,ESG,추월,8,6.878928
4,ESG,AllianceGSIA,8,6.878928
...,...,...,...,...
1333,ESG,운영,1,0.550991
1334,ESG,회계,1,0.345139
1335,ESG,사업장,1,0.310850
1336,ESG,오염,1,0.288627


In [14]:
report_esg_pmi_data_sort.to_csv('report_esg_pmi.csv', index=False, encoding="utf-8-sig")

In [6]:
report_esg_pmi_data_sort[:50]

NameError: name 'report_esg_pmi_data_sort' is not defined

In [2]:
result = pd.read_csv('report_esg_pmi.csv')

In [3]:
result[0:50]

,noun1,noun2,co-occurence,pmi
0,ESG,향조,9,6.996711
1,ESG,출하,8,6.878928
2,ESG,정한,8,6.878928
3,ESG,추월,8,6.878928
4,ESG,AllianceGSIA,8,6.878928
5,ESG,제종,8,6.878928
6,ESG,탄,7,6.745397
7,ESG,포부,5,6.408925
8,ESG,레,5,6.408925
9,ESG,KESG,5,6.408925


In [4]:
result[51:100]

,noun1,noun2,co-occurence,pmi
51,ESG,산출,190,5.919376
52,ESG,국무,9,5.898099
53,ESG,Engagement,12,5.898099
54,ESG,복제,6,5.898099
55,ESG,증가세,3,5.898099
56,ESG,환산,24,5.898099
57,ESG,저작물,6,5.898099
58,ESG,멘,6,5.898099
59,ESG,대여,6,5.898099
60,ESG,Watch,18,5.898099


In [5]:
result[100:150]

,noun1,noun2,co-occurence,pmi
100,ESG,선언,9,5.610417
101,ESG,영,9,5.610417
102,ESG,종합,35,5.582246
103,ESG,글로벌,70,5.582246
104,ESG,실패,24,5.579645
105,ESG,점수,493,5.531936
106,ESG,리포트,4,5.492634
107,ESG,견조,2,5.492634
108,ESG,대대,4,5.492634
109,ESG,자리매김,4,5.492634
